In [1]:
from transformers import RobertaModel,RobertaTokenizer
from utils import *
import os
os.environ['TRANSFORMERS_OFFLINE']='1'

/home/lyt/miniconda3/envs/DocRED/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device(7)

In [3]:
roberta_model = RobertaModel.from_pretrained('roberta-base').to(device)
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
import sys
sys.path.append('../..')
from kge.model import KgeModel
from kge.util.io import load_checkpoint


In [5]:
kg_type = 'half'
hops = 'webqsp_half'
checkpoint_file = '../../pretrained_models/embeddings/ComplEx_fbwq_' + kg_type + '/checkpoint_best.pt'
kge_checkpoint = load_checkpoint(checkpoint_file)
kge_model = KgeModel.create_from(kge_checkpoint)
kge_model.eval()
e = getEntityEmbeddings(kge_model, hops)

Loading configuration of dataset fbwq_half...
Setting complex.relation_embedder.dropout to 0, was set to -0.4746062345802784.
Loading half entity_ids.del


In [6]:
from torch.utils.data import Dataset, DataLoader
from dataloader import DatasetMetaQA, DataLoaderMetaQA


In [7]:
data_path = '../../data/QA_data/WebQuestionsSP/qa_train_webqsp.txt'
entity2idx, idx2entity, embedding_matrix = prepare_embeddings(e)
data = process_text_file(data_path, split=False)
print('Train file processed, making dataloader')
# word2ix,idx2word, max_len = get_vocab(data)
# hops = str(num_hops)
device = torch.device(device)
dataset = DatasetMetaQA(data, e, entity2idx)
data_loader = DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

Train file processed, making dataloader


loading file https://huggingface.co/roberta-base/resolve/main/vocab.json from cache at /data/lyt/transformers/d3ccdbfeb9aaa747ef20432d4976c32ee3fa69663b379deb253ccfce2bb1fdc5.d67d6b367eb24ab43b08ad55e014cf254076934f71d832bbab9ad35644a375ab
loading file https://huggingface.co/roberta-base/resolve/main/merges.txt from cache at /data/lyt/transformers/cafdecc90fcab17011e12ac813dd574b4b3fea39da6dd817813efa010262ff3f.5d12962c5ee615a4c803841266e9c3be9a691a924f72d395d3a6c6c81157788b
loading file https://huggingface.co/roberta-base/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/roberta-base/resolve/main/tokenizer_config.json from cache at None
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'Be

In [8]:
a = next(iter(data_loader))

In [9]:
question_tokenized = a[0].to(device)
attention_mask = a[1].to(device)
positive_head = a[2].to(device)
positive_tail = a[3].to(device)  

In [10]:
roberta_tokenizer.decode(question_tokenized[0].tolist(),skip_special_tokens=True)

' what is the name of justin bieber brother NE '

In [ ]:
question_tokenized[0]

In [12]:
test = roberta_tokenizer.encode('There is a test')
roberta_tokenizer.decode(test)

'<s>There is a test</s>'

In [13]:
device


device(type='cuda', index=7)

In [14]:
question_tokenized

tensor([[    0,    99,    16,     5,   766,     9,    95,   179,   741,   324,
          1943,  2138, 12462,  1437,     2,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1]], device='cuda:7')

In [15]:
roberta_model(question_tokenized, attention_mask=attention_mask)

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.0658,  0.0565, -0.0207,  ..., -0.1221, -0.0829,  0.0122],
         [ 0.0497, -0.5093,  0.1265,  ..., -0.0039,  0.1520,  0.0716],
         [ 0.3459,  0.0347,  0.0264,  ..., -0.2519,  0.2735,  0.1994],
         ...,
         [ 0.0700, -0.0940,  0.1141,  ..., -0.0928, -0.1418,  0.1312],
         [ 0.0700, -0.0940,  0.1141,  ..., -0.0928, -0.1418,  0.1312],
         [ 0.0700, -0.0940,  0.1141,  ..., -0.0928, -0.1418,  0.1312]]],
       device='cuda:7', grad_fn=<NativeLayerNormBackward0>), pooler_output=tensor([[ 2.1804e-03, -2.1176e-01, -2.1001e-01, -6.5291e-02,  1.3897e-01,
          2.1853e-01,  2.6736e-01, -7.9871e-02, -8.8498e-02, -1.5445e-01,
          2.2400e-01, -2.3583e-02, -9.8725e-02,  9.0742e-02, -1.3481e-01,
          4.9209e-01,  2.2070e-01, -4.5514e-01,  3.2645e-02, -2.6803e-02,
         -2.6233e-01,  4.7910e-02,  4.6291e-01,  3.2811e-01,  1.0480e-01,
          6.7569e-02, -1.4618e-01, -4.8004e-02,  1